# Binary classification of stork nest images

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sschmutz/stork-net/blob/master/scripts/05_binary-classification.ipynb)  
To use Google Colab, click on the link above and then change the Runtime type to Python 3 under "Runtime" - "Change runtime type". And for faster computation select GPU under "Hardware accelerator".

Code is adapted from the [TensorFlow Tutorial on Image classification](https://www.tensorflow.org/tutorials/images/classification).

The goal is to classify images of a stork nest in two categories, if a stork is present or not. The images were collected from a publicly available [webcam](https://www.berner-storch.ch/webcam/) and manually labeled.

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt

## Load images
Labeled images are already split and [available on GitHub](https://github.com/sschmutz/stork-net-dataset).
The full dataset will be downloaded with the following comand, this enables us to use this notebook in google colab.

In [ ]:
data_dir = tf.keras.utils.get_file(origin="https://github.com/sschmutz/stork-net-dataset/archive/master.zip", fname="stork-net-dataset-master.zip", extract=True)
data_dir = pathlib.Path(os.path.splitext(data_dir)[0])

train_dir = pathlib.Path(data_dir, "2019_train", "train")
validation_dir = pathlib.Path(data_dir, "2019_train", "validation")

In [ ]:
num_0_stork_train = len(list(train_dir.glob("0_stork/*.jpg")))
num_1_stork_train = len(list(train_dir.glob("1_stork/*.jpg")))
num_2_stork_train = len(list(train_dir.glob("2_stork/*.jpg")))
num_3_stork_train = len(list(train_dir.glob("3_stork/*.jpg")))

num_0_stork_val = len(list(validation_dir.glob("0_stork/*.jpg")))
num_1_stork_val = len(list(validation_dir.glob("1_stork/*.jpg")))
num_2_stork_val = len(list(validation_dir.glob("2_stork/*.jpg")))
num_3_stork_val = len(list(validation_dir.glob("3_stork/*.jpg")))

total_train = len(list(train_dir.glob("*/*.jpg")))
total_val = len(list(validation_dir.glob("*/*.jpg")))

class_names = np.array([item.name for item in train_dir.glob("*")])

In [ ]:
# I'm not sure if the numbers have to be divisible by the batch size.
# Currently the batch size is chosen to be divisible by total_train (280) and total_val (60)
batch_size = 20 
epochs = 15
img_height = 480
img_width = 640
channels = 3 #set to 1 if greyscale is used

Data augmentation can be defined already inside ***ImageDataGenerator()***, see the respective section on the [keras website](https://keras.io/api/preprocessing/image/).

In [ ]:
# The 1./255 is to convert from uint8 to float32 in range [0,1]
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

Should we change the color-images to grayscale? This way one can maybe use images from the infrared camera at night.
This could be done in ***flow_from_directory()***, just define following parameter: ***color_mode="grayscale"*** (default is "rgb").

If we use a multiclass-classification problem later on, we can define ***class_mode="categorical"***. Labels will be automatically be 2D one-hot encoded.

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(img_height, img_width),
                                                           class_mode="binary",
                                                           classes = list(class_names))

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_height, img_width),
                                                              class_mode="binary",
                                                              classes = list(class_names))

In [ ]:
sample_training_images, _ = next(train_data_gen)
sample_validation_images, _ = next(val_data_gen)

# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()
    
plotImages(sample_training_images[:5])

In [ ]:
class_names = ["0_stork", "1_stork", "2_stork", "3_stork"]

plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(sample_training_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    #plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## Create and train model

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding="same", activation="relu", input_shape=(img_height, img_width, channels)),
    MaxPooling2D(),
    Conv2D(32, 3, padding="same", activation="relu"),
    MaxPooling2D(),
    Conv2D(64, 3, padding="same", activation="relu"),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(4)
])

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss=history.history["loss"]
val_loss=history.history["val_loss"]

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
predictions = model.predict(val_data_gen)

In [ ]:
predictions

In [ ]:
predictions.shape

In [ ]:
predictions_prob = tf.round(tf.nn.sigmoid(predictions))
predictions_prob.shape

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(24, 40))
axes = axes.flatten()
for img, ax, i in zip(sample_validation_images, axes, range(60)):
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(round(predictions.item(i)))
plt.tight_layout()
plt.show()